In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 Vertex AI 预测开发 TensorFlow Serving

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/prediction/get_started_with_tf_serving.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在 Colab 中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fprediction%2Fget_started_with_tf_serving.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在 Colab Enterprise 中打开
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/prediction/get_started_with_tf_serving.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在 Workbench 中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/prediction/get_started_with_tf_serving.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在 GitHub 上查看
    </a>
  </td>
</table>

## 概述

本教程演示了如何使用`TensorFlow Serving`服务二进制文件从`Vertex AI Endpoint`中提供预测。

了解更多关于[从自定义训练模型中获取预测](https://cloud.google.com/vertex-ai/docs/predictions/get-predictions)。

### 目标

在本教程中，您将学习如何在 `Vertex AI Endpoint` 资源上使用 `Vertex AI Prediction` 和 `TensorFlow Serving` 服务二进制。

本教程使用以下 Vertex AI 服务和资源：

- `Vertex AI Prediction`
- `Vertex AI Batch Prediction`
- `Vertex AI Models`
- `Vertex AI Endpoints`

您将执行以下步骤：

- 从 TensorFlow Hub 下载预训练的图像分类模型。
- 创建一个用于接收压缩图像数据并输出预处理后的解压缩数据以用于模型输入的服务函数。
- 将 TensorFlow Hub 模型和服务函数上传为 Vertex AI 模型资源。
- 创建一个 Vertex AI Endpoint 资源。
- 使用 `TensorFlow Serving` 服务二进制将模型资源部署到端点资源上。
- 使用部署的模型进行在线预测。
- 使用模型进行批量预测。

数据集

本教程使用了来自TensorFlow Hub的预训练图像分类模型，该模型是在ImageNet数据集上训练的。

了解更多关于[ResNet V2预训练模型](https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5)。

### 成本

本教程使用 Google Cloud 的可计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [价格计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用情况生成成本估算。

开始吧

### 为Python安装Vertex AI SDK和其他必要软件包

In [ ]:
! pip3 install --upgrade google-cloud-aiplatform \
                        google-cloud-pipeline-components \
                        tensorflow==2.15.1 \
                        tensorflow-hub -q

重新启动运行时（仅适用于Colab）

要使用新安装的软件包，您必须在Google Colab上重新启动运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ 内核将重新启动。在继续下一步之前，请等待完成。⚠️</b>
</div>

### 验证您的笔记本环境（仅限Colab）

在谷歌Colab上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置 Google Cloud 项目信息

了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

### 创建一个云存储桶

创建一个存储桶，用于存储中间产物，比如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的存储桶尚不存在：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

### 导入库

In [ ]:
import json

import tensorflow as tf
import tensorflow_hub as hub
from google.cloud import aiplatform

初始化用于Python的Vertex AI SDK

为您的项目和对应的存储桶初始化Python的Vertex AI SDK。
要开始使用Vertex AI，您必须拥有一个现有的Google Cloud项目并[启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

设置硬件加速器

您可以为训练和预测设置硬件加速器。

将变量`DEPLOY_GPU/DEPLOY_NGPU`设置为使用支持 GPU 的容器映像和分配给虚拟机实例的 GPU 数量。例如，要使用一个分配了 4 个 Nvidia Telsa T4 GPU 的 GPU 容器映像来运行每个 VM，您可以指定：

    (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_T4, 4)

否则，指定`(None, None)`以在 CPU 上运行一个容器映像。

了解有关[您所在地区的硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

**注意**：TF 2.3 之前的版本对 GPU 支持在这个教程中无法加载自定义模型。这是一个已知问题，在 TF 2.3 中已得到解决。这是由于在服务函数中生成的静态图操作所导致。如果您在自己的自定义模型上遇到此问题，请使用支持 GPU 的 TF 2.3 容器映像。

In [ ]:
DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### 设置机器类型

接下来，设置用于预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置用于预测的VM的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB内存。
     - `n1-highmem`：每个vCPU 6.5GB内存
     - `n1-highcpu`：每个vCPU 0.9 GB内存
 - `vCPUs`：数量为\[2, 4, 8, 16, 32, 64, 96\]

**注意**：您还可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU。

In [ ]:
MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", DEPLOY_COMPUTE)

### 启用 Artifact Registry API

您必须为您的项目启用Artifact Registry API服务。

了解有关[启用服务](https://cloud.google.com/artifact-registry/docs/enable-service)的更多信息。

In [ ]:
! gcloud services enable artifactregistry.googleapis.com

创建私有的Docker仓库

您的第一步是在Google Artifact Registry中创建自己的Docker仓库。

1. 运行`gcloud artifacts repositories create`命令，在您的地区创建一个新的Docker仓库，并添加描述为“docker仓库”。

2. 运行`gcloud artifacts repositories list`命令来验证您的仓库是否已创建。

In [ ]:
PRIVATE_REPO = "my-docker-repo"

! gcloud artifacts repositories create {PRIVATE_REPO} --repository-format=docker --location={LOCATION} --description="Docker repository"

! gcloud artifacts repositories list

### 配置身份验证到您的私有存储库

在推送或拉取容器镜像之前，配置Docker使用`gcloud`命令行工具来对您所在地区的`Artifact Registry`进行身份验证请求。

In [ ]:
! gcloud auth configure-docker {LOCATION}-docker.pkg.dev --quiet

用于服务的容器（Docker）镜像

设置用于服务预测的TensorFlow Serving Docker容器镜像。

1. 从Docker Hub拉取相应的用于TF Serving的CPU或GPU Docker镜像。
2. 为图像创建一个标记，以在Artifact Registry中注册图像。
3. 使用Artifact Registry注册图像。

了解更多关于[TensorFlow Serving](https://www.tensorflow.org/tfx/serving/docker)。

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules

# Executes in Vertex AI Workbench
if DEPLOY_GPU:
    DEPLOY_IMAGE = (
        f"{LOCATION}-docker.pkg.dev/"
        + PROJECT_ID
        + f"/{PRIVATE_REPO}"
        + "/tf_serving:gpu"
    )
    TF_IMAGE = "tensorflow/serving:2.5.4-gpu"
else:
    DEPLOY_IMAGE = (
        f"{LOCATION}-docker.pkg.dev/"
        + PROJECT_ID
        + f"/{PRIVATE_REPO}"
        + "/tf_serving:cpu"
    )
    TF_IMAGE = "tensorflow/serving:2.5.4"

if not IS_COLAB:
    if DEPLOY_GPU:
        ! sudo docker pull tensorflow/serving:2.5.4-gpu
    else:
        ! sudo docker pull tensorflow/serving:2.5.4

    ! docker tag $TF_IMAGE $DEPLOY_IMAGE
    ! docker push $DEPLOY_IMAGE
else:
    # install docker daemon
    ! apt-get -qq install docker.io

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

在Colab中执行

In [ ]:
%%bash -s $IS_COLAB $DEPLOY_IMAGE $TF_IMAGE
if [ $1 == "False" ]; then
  exit 0
fi
set -x
dockerd -b none --iptables=0 -l warn &
for i in $(seq 5); do [ ! -S "/var/run/docker.sock" ] && sleep 2 || break; done
docker pull $3
docker tag $3 $2
docker push $2
kill $(jobs -p)

从TensorFlow Hub获取预训练模型

为演示目的，本教程使用了从TensorFlow Hub（TFHub）获取的预训练模型，然后将其上传到Vertex AI模型资源中。一旦您创建了Vertex AI模型资源，该模型就可以部署到Vertex AI终端资源中。

下载预训练模型

首先，从TensorFlow Hub下载预训练模型。该模型将作为一个TF.Keras层下载。为了完成模型，在本示例中，您将使用下载的TFHub模型创建一个`Sequential()`模型，并指定模型的输入形状。

In [ ]:
tfhub_model = tf.keras.Sequential(
    [hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5")]
)

tfhub_model.build([None, 224, 224, 3])

tfhub_model.summary()

### 保存模型工件

此时，模型已经在内存中。接下来，您需要将模型工件保存到云存储位置。

**注意：** 对于 TF 服务，MODEL_DIR 必须以一个数字结尾的子文件夹，例如，1。

In [ ]:
MODEL_DIR = BUCKET_URI + "/model/1"
tfhub_model.save(MODEL_DIR)

## 上传模型进行服务

接下来，您需要将从自定义作业中创建的TF.Keras模型上传到Vertex AI模型服务，这将为您的自定义模型创建一个模型资源。在上传过程中，您需要定义一个服务函数，将数据转换为模型所需的格式。如果您将编码数据发送到Vertex AI，您的服务函数将确保在数据被传递到模型之前，在模型服务器上对数据进行解码。

### 服务函数如何工作

当您向在线预测服务器发送请求时，该请求会被一个HTTP服务器接收。HTTP服务器从HTTP请求内容主体中提取出预测请求。提取出的预测请求会被转发给服务函数。对于谷歌预构建的预测容器，请求内容将作为`tf.string`传递给服务函数。

服务函数包括两个部分：

- `预处理函数`：
  - 将输入（`tf.string`）转换为底层模型（动态图）所需的输入形状和数据类型。
  - 执行训练底层模型期间执行的相同数据预处理 -- 例如，归一化，缩放等。
- `后处理函数`：
  - 将模型输出转换为接收应用程序预期的格式 -- 例如，压缩输出。
  - 为接收应用程序打包输出 -- 例如，添加标题，制作JSON对象等。

预处理和后处理函数都会转换为静态图，并与模型融合。从底层模型得到的输出会传递给后处理函数。后处理函数将转换/打包后的输出传递回HTTP服务器。HTTP服务器将输出作为HTTP响应内容返回。

在为TF.Keras模型构建服务函数时，需要考虑一个因素，即它们作为静态图运行。这意味着您不能使用需要动态图的TF图操作。如果这样做，您会在服务函数的编译过程中遇到错误，指示您正在使用不受支持的EagerTensor。

### 图像数据的服务功能

#### 预处理

为了将图像传递给预测服务，您需要将压缩的（例如JPEG）图像字节编码为base64——这样在通过网络传输二进制数据时可以确保内容不会被修改。由于部署的模型期望输入数据为原始（未压缩）字节，您需要确保base64编码数据被转换回原始字节，然后预处理以匹配模型的输入要求，然后将其作为输入传递给部署的模型。

为了解决这个问题，您需要定义一个服务函数（`serving_fn`）并将其附加到模型作为预处理步骤。添加一个`@tf.function`装饰器，以便将服务函数与底层模型融合在一起（而不是在CPU上游处理）。

当您发送预测或解释请求时，请求的内容会被base64解码为一个Tensorflow字符串（`tf.string`），然后传递给服务函数（`serving_fn`）。服务函数会将`tf.string`预处理为原始（未压缩）的numpy字节（`preprocess_fn`）以匹配模型的输入要求：

- `io.decode_jpeg`- 解压缩返回为带有三个通道（RGB）的Tensorflow张量的JPG图像。
- `image.convert_image_dtype` - 将整数像素值更改为float 32，并将像素数据重新缩放在0到1之间。
- `image.resize` - 调整图像大小以匹配模型的输入形状。

在这一点上，数据可以通过具体函数传递给模型（`m_call`）。服务函数是一个静态图，而模型是一个动态图。具体函数执行从服务函数到模型的输入数据编组的任务，并将模型的预测结果从模型返回到服务函数。

In [ ]:
CONCRETE_INPUT = "numpy_inputs"


def _preprocess(bytes_input):
    decoded = tf.io.decode_jpeg(bytes_input, channels=3)
    decoded = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize(decoded, size=(224, 224))
    return resized


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):
    decoded_images = tf.map_fn(
        _preprocess, bytes_inputs, dtype=tf.float32, back_prop=False
    )
    return {
        CONCRETE_INPUT: decoded_images
    }  # User needs to make sure the key matches model's input


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_fn(bytes_inputs):
    images = preprocess_fn(bytes_inputs)
    prob = m_call(**images)
    return prob


m_call = tf.function(tfhub_model.call).get_concrete_function(
    [tf.TensorSpec(shape=[None, 224, 224, 3], dtype=tf.float32, name=CONCRETE_INPUT)]
)

tf.saved_model.save(tfhub_model, MODEL_DIR, signatures={"serving_default": serving_fn})

获取服务功能签名

您可以通过重新加载模型到内存，并查询与每一层对应的签名来获取模型输入和输出层的签名。

为了您的目的，您需要服务功能的签名。为什么呢？当您将我们的数据作为HTTP请求数据包发送进行预测时，图像数据是Base64编码的，而我们的TF.Keras模型需要numpy输入。您的服务函数会将Base64转换为numpy数组。

在进行预测请求时，您需要将请求路由到服务函数而不是模型，因此您需要知道服务函数的输入层名称--您在后续进行预测请求时会使用。

In [ ]:
loaded = tf.saved_model.load(MODEL_DIR)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)

### 将 TensorFlow Hub 模型上传到 Vertex AI 模型资源

最后，您将 TFHub 模型和服务函数中的模型文件上传到 Vertex AI 模型资源中。由于您正在使用非 Google 预构建的服务二进制文件 -- 即 TensorFlow Serving，您需要指定以下额外的服务配置设置：

- `serving_container_command`：要启动的服务二进制文件（HTTP 服务器）。
- `serving_container_args`：要传递给服务二进制文件的参数。对于 TensorFlow Serving，需要的参数是：
    - `--model_name`：要分配给模型的可读名称。
    - `--model_base_name`：在容器中存储模型文件的位置。 Vertex 服务会将变量 $(AIP_STORAGE_URI) 设置为服务在容器中安装模型文件的位置。
    - `--rest_api_port`：用于发送基于 REST 的预测请求的端口。可以是 8080 或 8501（TensorFlow Serving 的默认值）。
    - `--port`：用于发送基于 gRPC 的预测请求的端口。对于 TensorFlow Serving，应该是 8500。
- `serving_container_health_route`：服务定期检查以验证服务二进制是否正在运行的 URL。对于 TensorFlow Serving，这是 /v1/models/\<model_name\>。
- `serving_container_predict_route`：服务用于路由基于 REST 的预测请求的 URL。对于 TF Serving，这是 /v1/models/[model_name]:predict。
- `serving_container_ports`：HTTP 服务器监听请求的端口列表。

将模型上传到 Vertex 模型资源会返回一个长时间运行的操作，因为可能需要一些时间。

**注意：** 您需要删除模型路径中的结尾数字子文件夹（例如，/1）才能进行上传。Vertex 服务会上传包含模型文件的子文件夹上方的父文件夹 -- 这是 TensorFlow Serving 二进制文件所期望的。

**注意：** 当您将模型文件上传到 Vertex AI 模型资源时，您需要指定相应的部署容器镜像。

In [ ]:
MODEL_NAME = "example_"

model = aiplatform.Model.upload(
    display_name="example_",
    artifact_uri=MODEL_DIR[:-2],
    serving_container_image_uri=DEPLOY_IMAGE,
    serving_container_health_route="/v1/models/" + MODEL_NAME,
    serving_container_predict_route="/v1/models/" + MODEL_NAME + ":predict",
    serving_container_command=["/usr/bin/tensorflow_model_server"],
    serving_container_args=[
        "--model_name=" + MODEL_NAME,
        "--model_base_path=" + "$(AIP_STORAGE_URI)",
        "--rest_api_port=8080",
        "--port=8500",
        "--file_system_poll_wait_seconds=31540000",
    ],
    serving_container_ports=[8080],
)

print(model)

## 创建一个顶点 AI 终点资源

您可以使用 `Endpoint.create()` 方法来创建一个顶点 AI 终点资源。最少要指定端点的显示名称。可选地，您可以指定项目和位置（区域）；否则，设置将继承您在使用 `init()` 方法初始化顶点 AI SDK 时设置的值。

在此示例中，指定了以下参数：

- `display_name`: 终点资源的人类可读名称。
- `project`: 您的项目 ID。
- `location`: 您的区域。
- `labels`:（可选）以键值对形式提供给终点的用户自定义元数据。

此方法返回一个顶点 AI 终点对象。

了解更多关于[Vertex AI 终点](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)。

In [ ]:
endpoint = aiplatform.Endpoint.create(
    display_name="example_",
    project=PROJECT_ID,
    location=LOCATION,
    labels={"your_key": "your_value"},
)

print(endpoint)

## 部署模型到端点

您可以将一个或多个Vertex AI模型资源实例部署到同一个端点。每个部署的模型资源都有自己的服务二进制文件的部署容器。

**注意：** 在此示例中，您在上一步中指定了TFHub模型的部署容器，以上传模型工件到Vertex AI模型资源。

在下一个示例中，您将将Vertex AI模型资源部署到Vertex AI端点资源。模型资源已经有一个定义好的容器镜像用于部署。要部署，您需要指定以下额外的配置设置：

- 机器类型。
- （如果有的话）GPU 的类型和数量。
- VM 实例的静态、手动或自动扩展。

在本示例中，您将使用最少量的指定参数部署模型，具体如下：

- `model`：Vertex AI模型资源。
- `deployed_model_displayed_name`：部署模型实例的人类可读名称。
- `machine_type`：每个VM实例的机器类型。

由于资源配置的要求，这可能需要几分钟的时间。

In [ ]:
response = endpoint.deploy(
    model=model,
    deployed_model_display_name="example_",
    machine_type=DEPLOY_COMPUTE,
)

print(endpoint)

准备用于预测的测试数据

接下来，您将一个压缩的JPEG图片加载到内存中，然后对其进行base64编码。为了演示目的，您将使用来自Flowers数据集的一张图片。

In [ ]:
! gsutil cp gs://cloud-ml-data/img/flower_photos/daisy/100080576_f52e8ee070_n.jpg test.jpg

In [ ]:
import base64

with open("test.jpg", "rb") as f:
    data = f.read()
b64str = base64.b64encode(data).decode("utf-8")

### 进行预测

现在您的模型资源已部署到一个端点资源，您可以通过向端点资源发送预测请求来进行在线预测。

#### 请求

在这个示例中，由于您的测试项目存储在一个Cloud Storage存储桶中，您可以使用 `tf.io.gfile.Gfile()` 打开并读取图像的内容。为了将测试数据传递给预测服务，您需要将字节编码成base64格式 -- 这样可以确保在通过网络传输二进制数据时内容不会被修改。

每个实例的格式如下：

    { serving_input: { 'b64': base64编码的字节 } }

由于 `predict()` 方法可以接受多个项目(实例)，请将您的单个测试项目作为一个包含单个测试项目的列表发送。

#### 响应

从 `predict()` 调用中返回的是一个Python字典，其中包含以下条目：

- `ids`: 每个预测请求的内部分配的唯一标识符。
- `predictions`: 每个类标签的预测置信度，介于0到1之间。
- `deployed_model_id`: 执行预测的Vertex AI部署的 `Model` 资源的标识符。

In [ ]:
# The format of each instance should conform to the deployed model's prediction input schema.
instances = [{serving_input: {"b64": b64str}}]

prediction = endpoint.predict(instances=instances)

print(prediction)

## 批量预测介绍

批量预测提供了对大量预测请求进行离线批处理的能力。资源仅在批处理过程中进行配置，然后在批请求完成时取消配置。结果存储在云存储中，与在线预测不同，在线预测的结果以HTTP响应数据包返回。

批处理作业的输入格式取决于您的模型服务器支持的格式。首先，您的模型服务器中的Web服务器必须支持 JSONL 格式，该格式由Web服务器转换为模型输入接口直接支持或服务函数接口支持的格式之一。对于批量预测，这种 JSONL 格式被称为“pivot”格式。

### 批量预测作业的输入格式

批处理服务器接受以下输入格式：

- JSONL
- CSV
- TFRecords
- 文件列表

### Pivot格式

批处理服务器将输入格式转换为`pivot`（JSONL）格式，如下所示：

**JSONL**

每个输入行（请求）应包含一个且仅一个有效的 JSON 值。

    {"values": [1, 2, 3, 4], "key": 1}
    {"values": [5, 6, 7, 8], "key": 2}

批处理服务器使用相同格式生成 pivot 数据。然后生成的 pivot 数据被包装成一个负载请求：

    {"instances": [
      {"values": [1, 2, 3, 4], "key": 1},
      {"values": [5, 6, 7, 8], "key": 2}
    ]}

**CSV**

第一行中的 CSV 标头始终被忽略。字符串字段需要明确地用双引号括起来，否则行将被丢弃，并且将输出解析错误消息到错误文件。非引用值始终会转换为浮点数。

    col1,col2,col3
    1,3,"cat1"
    2,4,"cat2"

批处理服务器将每个输入行（请求）转换为一个 JSON 数组。

    {"instances": [
     [1.0,3.0,"cat1"],
     [2.0,4.0,"cat2"]
    ]}
    
**BigQuery**

每行转换为一个 JSON 数组。例如：

    [1.0,3.0,"cat1"]
    [2.0,4.0,"cat2"]
    
批处理服务器使用相同格式生成 pivot 数据。然后生成的 pivot 数据被包装成一个负载请求：

    {"instances": [
     [1.0,3.0,"cat1"],
     [2.0,4.0,"cat2"]
    ]}

**TFRecords**

TFRecord 文件中的实例由 apache_beam.io.tfrecordio 模块以二进制形式读取。然后将二进制对象序列化为 ASCII 字符串。预测服务器负责知道解码器以恢复实例。

    {"instances": [
     {"b64","b64EncodedASCIIString"},
     {"b64","b64EncodedASCIIString"}
    ]}

**文件列表**

FileList 格式包含文件列表。在“文件列表”文件中，每行指定一个文件路径，指定为云存储位置。

    gs://my-bucket/file1.txt
    gs://my-bucket/file2.txt

批处理服务器将文件作为二进制数据读取。二进制对象被序列化为 ASCII 字符串。

    {"instances": [
     {"b64","b64EncodedASCIIString"},
     {"b64","b64EncodedASCIIString"}
    ]}

## 创建批量输入文件

接下来，创建一个批量输入文件，将其存储在本地的云存储桶中。对于自定义模型，您需要以JSONL格式格式化批量输入文件。 JSONL文件中的每个JSON对象条目的格式与您为在线预测请求指定的格式相同。

换句话说，在线预测和批量预测使用相同的预测请求格式。不同之处在于，在在线预测中，您使用SDK方法`predict()`将请求作为内存字典对象传递。对于批量预测，您将每个预测请求（字典条目）写成一个JSON对象，每行一个。

字典包含以下键/值对：

- `input_name`：底层模型的输入层名称。
- `'b64'`：指示内容为base64编码的键。
- `content`：压缩的JPG图像字节作为base64编码的字符串。

预测请求中的每个实例是以下形式的字典条目：

                        {serving_input: {'b64': content}}

要将图像数据传递给预测服务，您需要将字节编码为base64-- 这样可以在通过网络传输二进制数据时使内容免受修改。

In [ ]:
# For demonstration purposes, you write the same image (instance[0]) request twice to the JSONL file.
# You receive back two predictions, one for each instance.

with open("test.jsonl", "w") as f:
    json.dump(instances[0], f)
    f.write("\n")
    json.dump(instances[0], f)

! gsutil cp test.jsonl {BUCKET_URI}/test.jsonl

## 进行批量预测请求

现在您的模型资源已经训练完成，您可以通过调用 `batch_predict()` 方法进行批量预测，需要以下参数：

- `job_display_name`：批量预测作业的人类可读名称。
- `gcs_source`：一个或多个批请求输入文件的列表。
- `gcs_destination_prefix`：用于存储批量预测结果的Cloud Storage位置。
- `instances_format`：输入实例的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `predictions_format`：输出预测的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `machine_type`：用于训练的计算机类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：连接到工作进程副本的加速器数量。
- `sync`：如果设置为True，则调用将在等待异步批处理作业完成时被阻塞。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

batch_predict_job = model.batch_predict(
    job_display_name="example_",
    instances_format="jsonl",
    predictions_format="jsonl",
    model_parameters=None,
    gcs_source=f"{BUCKET_URI}/test.jsonl",
    gcs_destination_prefix=f"{BUCKET_URI}/results",
    machine_type=DEPLOY_COMPUTE,
    accelerator_type=DEPLOY_GPU,
    accelerator_count=DEPLOY_NGPU,
    starting_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    sync=False,
)

等待批量预测作业完成

接下来，等待批量作业完成。或者，您可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻塞直到批量预测作业完成。

In [ ]:
batch_predict_job.wait()

## 获取预测结果

接下来，从已完成的批量预测作业中获取结果。

结果将写入您在批量预测请求中指定的云存储输出桶中。 您可以调用iter_outputs()方法获取生成的结果的每个云存储文件的列表。 每个文件以JSON格式包含一个或多个预测请求：

- `instance`：预测请求。
- `prediction`：预测响应。

In [ ]:
bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

## 清理

要清理此项目中使用的所有Google Cloud资源，可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

或者，您可以删除在此教程中创建的各个资源。

In [ ]:
# set delete_bucket to True to delete your bucket
delete_bucket = False

# undeploy the model from the endpoint
endpoint.undeploy_all()

# delete the endpoint
endpoint.delete()

# delete the model
model.delete()

# delete the batch job
batch_predict_job.delete()

# delete the bucket
if delete_bucket:
    ! gsutil rm -rf {BUCKET_URI}